In [ ]:
# default_exp data.token_classification

In [ ]:
#all_slow

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.token_classification

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for token classification tasks (e.g., NER or named entity recognition, etc...).

In [ ]:
#export
import os, ast
from functools import reduce

from fastcore.all import *
from fastai.data.block import TransformBlock, Category, CategoryMap
from fastai.imports import *
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForTokenClassification, logging

from blurr.utils import BLURR
from blurr.data.core import HF_BaseInput, HF_BeforeBatchTransform, first_blurr_tfm

logging.set_verbosity_error()

In [ ]:
#hide_input
import pdb

from fastai.data.block import DataBlock, ColReader, ColSplitter
from fastai.data.core import DataLoader, DataLoaders, TfmdDL
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import *
from fastcore.test import *
from nbverbose.showdoc import show_doc

from blurr.utils import print_versions
from blurr.data.core import HF_TextBlock

os.environ["TOKENIZERS_PARALLELISM"] = "false"
print_versions('torch fastai transformers')

torch: 1.7.1
fastai: 2.5.0
transformers: 4.9.2


In [ ]:
#hide_input
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Token classification tokenization, batch transform, and DataBlock methods

Token classification tasks attempt to predict a class for each token.  The idea is similar to that in image segmentation models where the objective is to predict a class for each pixel.  Such models are common in building named entity recognition (NER) systems.

In [ ]:
# ensures these cols are represented as lists (rather than string)
df_converters = {'tokens': ast.literal_eval, 'labels': ast.literal_eval, 'nested-labels': ast.literal_eval}

path = Path('./')
germ_eval_df = pd.read_csv(path/'germeval2014_sample.csv', converters=df_converters); len(germ_eval_df)

1000

In [ ]:
#hide
# for idx, el in germ_eval_df.iterrows():
#     print (el['tokens'])
#     print (el['labels'])
#     print('---------------')

In [ ]:
labels = sorted(list(set([lbls for sublist in germ_eval_df.labels.tolist() for lbls in sublist])))
print(labels)

['B-LOC', 'B-LOCderiv', 'B-LOCpart', 'B-ORG', 'B-ORGpart', 'B-OTH', 'B-OTHderiv', 'B-OTHpart', 'B-PER', 'B-PERderiv', 'B-PERpart', 'I-LOC', 'I-LOCderiv', 'I-ORG', 'I-ORGpart', 'I-OTH', 'I-PER', 'O']


In [ ]:
model_cls = AutoModelForTokenClassification

# pretrained_model_name = "bert-base-multilingual-cased"
pretrained_model_name = 'roberta-base'
n_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=model_cls,
                                                                  config_kwargs={'num_labels': n_labels})
hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)

('roberta',
 transformers.models.roberta.configuration_roberta.RobertaConfig,
 transformers.models.roberta.tokenization_roberta_fast.RobertaTokenizerFast,
 transformers.models.roberta.modeling_roberta.RobertaForTokenClassification)

Below, we define a new class and transform for token classification targets/predictions.

In [ ]:
#export
class HF_TokenTensorCategory(TensorBase): pass

In [ ]:
#export
class HF_TokenCategorize(Transform):
    "Reversible transform of a list of category string to `vocab` id"
    
    def __init__(self, vocab=None, ignore_token=None, ignore_token_id=None):  
        self.vocab = None if vocab is None else CategoryMap(vocab)
        self.ignore_token = '[xIGNx]' if ignore_token is None else ignore_token
        self.ignore_token_id = CrossEntropyLossFlat().ignore_index if ignore_token_id is None else ignore_token_id
        
        self.loss_func, self.order = CrossEntropyLossFlat(ignore_index=self.ignore_token_id), 1

    def setups(self, dsets):
        if self.vocab is None and dsets is not None: self.vocab = CategoryMap(dsets)
        self.c = len(self.vocab)

    def encodes(self, labels):
        ids = [[self.vocab.o2i[lbl]] + [self.ignore_token_id]*(n_subtoks-1) for lbl, n_subtoks in labels] 
        return HF_TokenTensorCategory(reduce(operator.concat, ids))
    
    def decodes(self, encoded_labels): 
        return Category([(self.vocab[lbl_id]) for lbl_id in encoded_labels if lbl_id != self.ignore_token_id ])

`HF_TokenCategorize` modifies the fastai `Categorize` transform in a couple of ways.  First, it allows your targets to consist of a `Category` ***per*** token, and second, it uses the idea of an `ignore_token_id` to mask subtokens that don't need a prediction.  For example, the target of special tokens (e.g., pad, cls, sep) are set to `ignore_token_id` as are subsequent sub-tokens of a given token should more than 1 sub-token make it up.

In [ ]:
#export
def HF_TokenCategoryBlock(vocab=None, ignore_token=None, ignore_token_id=None):
    "`TransformBlock` for single-label categorical targets"
    return TransformBlock(type_tfms=HF_TokenCategorize(vocab=vocab, 
                                                       ignore_token=ignore_token,
                                                       ignore_token_id=ignore_token_id))

In [ ]:
show_doc(HF_TokenCategoryBlock)

<h4 id="HF_TokenCategoryBlock" class="doc_header"><code>HF_TokenCategoryBlock</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>HF_TokenCategoryBlock</code>(**`vocab`**=*`None`*, **`ignore_token`**=*`None`*, **`ignore_token_id`**=*`None`*)

`TransformBlock` for single-label categorical targets

**Parameters:**


 - **`vocab`** : *`<class 'NoneType'>`*, *optional*

 - **`ignore_token`** : *`<class 'NoneType'>`*, *optional*

 - **`ignore_token_id`** : *`<class 'NoneType'>`*, *optional*


Again, we define a custom class, `HF_TokenClassInput`, for the @typedispatched methods to use so that we can override how token classification inputs/targets are assembled, as well as, how the data is shown via methods like `show_batch` and `show_results`.

In [ ]:
#export
class HF_TokenClassInput(HF_BaseInput): pass

In [ ]:
#export
class HF_TokenClassBeforeBatchTransform(HF_BeforeBatchTransform):
    def __init__(
        self, 
        hf_arch, 
        hf_config, 
        hf_tokenizer, 
        hf_model,
        ignore_token_id=CrossEntropyLossFlat().ignore_index,
        max_length=None, 
        padding=True, 
        truncation=True, 
        is_split_into_words=True, 
        tok_kwargs={}, **kwargs
    ):
        super().__init__(hf_arch, hf_config, hf_tokenizer, hf_model,
                         max_length=max_length, padding=padding, truncation=truncation, 
                         is_split_into_words=is_split_into_words, tok_kwargs=tok_kwargs, **kwargs)

        self.ignore_token_id = ignore_token_id
        
    def encodes(self, samples):  
        samples = super().encodes(samples)
        if (len(samples[0]) == 1): return samples
        
        target_cls = type(samples[0][1])
        updated_samples = []
        
        # we assume that first target = the categories we want to predict for each token
        for s in samples:
            targ_len = len(s[1])
            idx_first_input_id = s[0]['special_tokens_mask'].tolist().index(0)
            targ_ids = target_cls([ self.ignore_token_id if (el == 1 or idx > targ_len) 
                                   else s[1][idx-idx_first_input_id].item() 
                                   for idx, el in enumerate(s[0]['special_tokens_mask']) ])

            updated_samples.append((s[0], targ_ids))
        
        return updated_samples

`HF_TokenClassBeforeBatchTransform` is used to exclude any of the target's tokens we don't want to include in the loss calcuation (e.g. padding, cls, sep, etc...).

In [ ]:
before_batch_tfm = HF_TokenClassBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                                     is_split_into_words=True, 
                                                     tok_kwargs={ 'return_special_tokens_mask': True })

blocks = (
    HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_TokenClassInput), 
    HF_TokenCategoryBlock(vocab=labels)
)

def get_y(inp):
    return [ (label, len(hf_tokenizer.tokenize(str(entity)))) for entity, label in zip(inp.tokens, inp.labels) ]

dblock = DataBlock(blocks=blocks, get_x=ColReader('tokens'), get_y=get_y, splitter=RandomSplitter())

Note in the example above we had to define a `get_y` in order to return both the entity we want to predict a category for, as well as, how many subtokens are used by the `hf_tokenizer` to represent it.  This is necessary for the input/target alignment discussed above.

In [ ]:
#hide
# dblock.summary(germ_eval_df)

In [ ]:
dls = dblock.dataloaders(germ_eval_df, bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b), b[0]['input_ids'].shape, b[1].shape

(2, torch.Size([4, 98]), torch.Size([4, 98]))

In [ ]:
#export
@typedispatch
def show_batch(
    x:HF_TokenClassInput, 
    y, 
    samples, 
    dataloaders, 
    ctxs=None, 
    max_n=6, 
    trunc_at=None, 
    **kwargs
):  
    # grab our tokenizer
    tfm = first_blurr_tfm(dataloaders, before_batch_tfm_class=HF_TokenClassBeforeBatchTransform) 
    hf_tokenizer = tfm.hf_tokenizer
    
    res = L()
    for inp, trg, sample in zip(x, y, samples):
        # recontstruct the string and split on space to get back your pre-tokenized list of tokens
        toks = hf_tokenizer.convert_ids_to_tokens(inp, skip_special_tokens=True)
        pretokenized_toks =  hf_tokenizer.convert_tokens_to_string(toks).split()

        res.append([f'{[ (tok, lbl) for idx, (tok, lbl) in enumerate(zip(pretokenized_toks, ast.literal_eval(sample[1]))) if (trunc_at is None or idx < trunc_at) ]}'])

    display_df(pd.DataFrame(res, columns=['token / target label'])[:max_n])
    return ctxs

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=10)

,token / target label
0,"[('Helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('S.', 'O'), ('593.', 'O'), ('Wink', 'B-OTH')]"
1,"[('NEWSru.ua', 'B-OTH'), ('/', 'O'), (':', 'O'), ('Политисполком', 'B-OTH'), ('СПУ', 'I-OTH'), ('отказал', 'I-OTH'), ('Морозу', 'I-OTH'), ('в', 'I-OTH'), ('отставке', 'I-OTH'), ('Die', 'O')]"


## Tests

The tests below to ensure the core DataBlock code above works for **all** pretrained token classification models available in Hugging Face.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained classification models you are working with ... and if any of your pretrained token classification models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
#hide
[ model_type for model_type in BLURR.get_models(task='TokenClassification') 
 if (not model_type.startswith('TF')) ]

['AlbertForTokenClassification',
 'BertForTokenClassification',
 'BigBirdForTokenClassification',
 'CamembertForTokenClassification',
 'CanineForTokenClassification',
 'ConvBertForTokenClassification',
 'DebertaForTokenClassification',
 'DebertaV2ForTokenClassification',
 'DistilBertForTokenClassification',
 'ElectraForTokenClassification',
 'FlaubertForTokenClassification',
 'FunnelForTokenClassification',
 'IBertForTokenClassification',
 'LayoutLMForTokenClassification',
 'LongformerForTokenClassification',
 'MPNetForTokenClassification',
 'MegatronBertForTokenClassification',
 'MobileBertForTokenClassification',
 'RoFormerForTokenClassification',
 'RobertaForTokenClassification',
 'SqueezeBertForTokenClassification',
 'XLMForTokenClassification',
 'XLMRobertaForTokenClassification',
 'XLNetForTokenClassification']

In [ ]:
#hide
pretrained_model_names = [
    'albert-base-v1',
    'bert-base-multilingual-cased',
    'camembert-base',
    'distilbert-base-uncased',
    'monologg/electra-small-finetuned-imdb',
    'flaubert/flaubert_small_cased',
    'huggingface/funnel-small-base',
    'allenai/longformer-base-4096',
    'microsoft/mpnet-base',
    'google/mobilebert-uncased',
    'roberta-base',
    'squeezebert/squeezebert-uncased',
    'xlm-mlm-en-2048',
    'xlm-roberta-base',
    'xlnet-base-cased'
]

In [ ]:
#hide
model_cls = AutoModelForTokenClassification
bsz = 2
seq_sz = 128

test_results = []
for model_name in pretrained_model_names:
    error=None
    
    print(f'=== {model_name} ===\n')
    
    hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(model_name, model_cls=model_cls)
    print(f'architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\n')
    
    # not all architectures include a native pad_token (e.g., gpt2, ctrl, etc...), so we add one here
    if (hf_tokenizer.pad_token is None): 
        hf_tokenizer.add_special_tokens({'pad_token': '<pad>'})  
        hf_config.pad_token_id = hf_tokenizer.get_vocab()['<pad>']
        hf_model.resize_token_embeddings(len(hf_tokenizer))   
    
    before_batch_tfm = HF_TokenClassBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                                         padding='max_length', 
                                                         max_length=seq_sz, 
                                                         is_split_into_words=True, 
                                                         tok_kwargs={ 'return_special_tokens_mask': True })

    blocks = (
        HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_TokenClassInput), 
        HF_TokenCategoryBlock(vocab=labels)
    )

    dblock = DataBlock(blocks=blocks, 
                       get_x=ColReader('tokens'),
                       get_y= lambda inp: [ (label, len(hf_tokenizer.tokenize(str(entity)))) 
                                           for entity, label in zip(inp.tokens, inp.labels) ],
                       splitter=RandomSplitter())
    
    dls = dblock.dataloaders(germ_eval_df, bs=bsz)
    b = dls.one_batch()
    
    try:
        print('*** TESTING DataLoaders ***\n')
        test_eq(len(b), 2)
        test_eq(len(b[0]['input_ids']), bsz)
        test_eq(b[0]['input_ids'].shape, torch.Size([bsz, seq_sz]))
        test_eq(len(b[1]), bsz)

        if (hasattr(hf_tokenizer, 'add_prefix_space')):
             test_eq(hf_tokenizer.add_prefix_space, True)
                
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'PASSED', ''))
        dls.show_batch(dataloaders=dls, max_n=2, trunc_at=10)
        
    except Exception as err:
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'FAILED', err))

=== albert-base-v1 ===

architecture:	albert
tokenizer:	AlbertTokenizerFast

*** TESTING DataLoaders ***



,token / target label
0,"[('helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('s.', 'O'), ('593.', 'O'), ('wink', 'B-OTH')]"
1,"[('in', 'O'), ('einer', 'O'), ('fernsehdiskussion', 'O'), ('traf', 'O'), ('er', 'O'), ('auf', 'O'), ('den', 'O'), ('kritiker', 'O'), ('alexander', 'B-PER'), ('walker', 'I-PER')]"


=== bert-base-multilingual-cased ===

architecture:	bert
tokenizer:	BertTokenizerFast

*** TESTING DataLoaders ***



,token / target label
0,"[('Helbig', 'B-OTH'), ('et', 'I-OTH'), ('al.', 'I-OTH'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('S.', 'O'), ('593.', 'O'), ('Wink', 'O'), ('&', 'B-OTH')]"
1,"[('Zugang', 'O'), ('und', 'O'), ('Engagement', 'O'), (':', 'O'), ('das', 'O'), ('eigentlich', 'O'), ('Neue', 'O'), ('an', 'O'), ('der', 'O'), ('Netz', 'O')]"


=== camembert-base ===

architecture:	camembert
tokenizer:	CamembertTokenizerFast

*** TESTING DataLoaders ***



...

,token / target label
0,"[('helbig', 'B-OTH'), ('et', 'I-OTH'), ('al', 'I-OTH'), ('.', 'O'), ('(', 'O'), ('1994', 'O'), (')', 'O'), ('s', 'O'), ('.', 'O'), ('593', 'B-OTH')]"
1,"[('mit', 'O'), ('der', 'O'), ('servicefrau', 'O'), ('verband', 'O'), ('bianca', 'B-PER'), ('offenbar', 'O'), ('eine', 'O'), ('art', 'O'), ('freundschaft', 'O'), ('-', 'O')]"


=== xlm-roberta-base ===

architecture:	xlm_roberta
tokenizer:	XLMRobertaTokenizerFast

*** TESTING DataLoaders ***



,token / target label
0,"[('(', 'O'), ('Standard', 'B-ORG'), ('Oil', 'I-ORG'), ('of', 'I-ORG'), ('New', 'I-ORG'), ('Jersey', 'I-ORG'), (')', 'O'), (',', 'O'), ('die', 'O'), ('ausgesprochen', 'O')]"
1,"[('NEWSru.ua', 'B-OTH'), ('/', 'O'), (':', 'O'), ('Политисполком', 'B-OTH'), ('СПУ', 'I-OTH'), ('отказал', 'I-OTH'), ('Морозу', 'I-OTH'), ('в', 'I-OTH'), ('отставке', 'I-OTH'), ('Die', 'O')]"


=== xlnet-base-cased ===

architecture:	xlnet
tokenizer:	XLNetTokenizerFast

*** TESTING DataLoaders ***



,token / target label
0,"[('(', 'O'), ('Standard', 'B-ORG'), ('Oil', 'I-ORG'), ('of', 'I-ORG'), ('New', 'I-ORG'), ('Jersey', 'I-ORG'), (')', 'O'), (',', 'O'), ('die', 'O'), ('ausgesprochen', 'O')]"
1,"[('Zugang', 'O'), ('und', 'O'), ('Engagement', 'O'), (':', 'O'), ('das', 'O'), ('eigentlich', 'O'), ('Neue', 'O'), ('an', 'O'), ('der', 'O'), ('Netz(', 'O')]"


In [ ]:
#hide_input
test_results_df = pd.DataFrame(test_results, columns=['arch', 'tokenizer', 'model_name', 'result', 'error'])
display_df(test_results_df)

,arch,tokenizer,model_name,result,error
0,albert,AlbertTokenizerFast,albert-base-v1,PASSED,
1,bert,BertTokenizerFast,bert-base-multilingual-cased,PASSED,
2,camembert,CamembertTokenizerFast,camembert-base,PASSED,
3,distilbert,DistilBertTokenizerFast,distilbert-base-uncased,PASSED,
4,electra,ElectraTokenizerFast,monologg/electra-small-finetuned-imdb,PASSED,
5,flaubert,FlaubertTokenizer,flaubert/flaubert_small_cased,PASSED,
6,funnel,FunnelTokenizerFast,huggingface/funnel-small-base,PASSED,
7,longformer,LongformerTokenizerFast,allenai/longformer-base-4096,PASSED,
8,mpnet,MPNetTokenizerFast,microsoft/mpnet-base,PASSED,
9,mobilebert,MobileBertTokenizerFast,google/mobilebert-uncased,PASSED,


## Summary

This module includes all the low, mid, and high-level API bits for token classification tasks data prep.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted index.ipynb.
